## Ejemplo de cálculo con lo básico

### Python in Elecrical Engineering: Computing Symmetrical Components. [link](http://my-other-life-as-programmer.blogspot.com.ar/2011/04/python-in-elecrical-engineering.html)

Fasores: vectores rotantes a una velocidad dad por la frecuencia de línea, $V_1$, $V_2$, $V_3$, para cada uno tenemos un módulo y un ángulo de fase.

El cálculo de las componentes simétricas, está dado por:

$E_0 = (\frac {1}{3}) (V_1 + V_2 + V_3)$, 

$E_1 = (\frac {1}{3}) (V_1 + aV_2 + a^2V_3)$,

$E_2 = (\frac {1}{3}) (V_1 + a^2V_2 + aV_3)$,

A es un operador fasor rotado 120°

$a = cos( 2\pi/3) + j sin(2\pi/3) =  e^{i2\pi/3}$



In [3]:
"""
file    symcomp.py
author  Ernesto P. Adorio, Ph.D.
UPDEPP (U.P. Clarkfield)
desc    basic symmetrical components for three phase phasors.
Phasors are tuples of the form (r, phi) where r and phi are the
magnitude(absolute value) and phi is the phase angle.
version 0.0.1 april 20, 2011
"""

#from math  import *
#from cmath import *

import math as mt
import cmath as cm

zTOL = 1.0e-8
DTOR = mt.pi / 180.0
RTOD = 180.0 / mt.pi



ONETWENTYRAD = 120 * DTOR
_a_  = cm.rect(1, 120 * mt.pi/ 180.0)
_a2_ = _a_.conjugate()

def dtor(degree):
    """
    Converts degree to radians.
    """
    return degree * DTOR

def rtod(radian):
    """
    Converts radians to degree.
    """
    return radian * RTOD

def  z2pair(z):
    """
    returns z in pair form.
    """ 
    return (z.real, z.imag)

def  pair2z(re, im):
    """
    returns a complex number from the components. 
    """
    return re + im*1j

def  polar2pair(v):
    """
    (r, phi) to (re, im) pair form.
    """
    # Special checking for zero imaginaries!
    # some computations result in an additional 0j
    r, phi = v
    if type(r) == type(1j):
        r =r.real
    if type(phi) == type(1j):
         phi = phi.real

    z = cm.rect(r, phi)
    return z.real, z.imag

def  pair2polar(re, im):
    return polar(pair2z(re,im))

def  zround(v):
    if abs(v[0]) < zTOL:
        return (0.0, 0.0)

def  a(v):
     """
     Applies the a operator to a phasor v in polar form.
     It adds a 120 degree to the phase of the phasor v.
     if v is real, the angle is zero.
     """
     if type(v) != type((0,0)):
        v = (v, 0)
     r     = abs(v[0])
     theta = v[1]

     newangle = theta + ONETWENTYRAD
     return (r * cos(newangle), r * sin(newangle))

def a2(v):
     """
     Applies the a operator to a phasor v in polar form.
     It adds a 120 degree to the phase of the phasor v.
     """
     if type(v) != type((0,0)):
        v = (v, 0)
     r     = abs(v[0])
     theta = v[1]

     newangle = theta + 2*ONETWENTYRAD
     return (r * cos(newangle), r * sin(newangle))


def  symcomp(v1, v2, v3):
     """
     Returns the symmetrical components of the three phasors v1, v2, v3
     which are in tuple (r, theta) form.
     """
     #Convert first to complex rectangular form.
     v1z  = cm.rect(v1[0], v1[1])
     v2z  = cm.rect(v2[0], v2[1])
     v3z  = cm.rect(v3[0], v3[1]) 

     av2  = _a_ * v2z
     a2v2 = _a2_ * v2z
     av3  = _a_* v3z
     a2v3 = _a2_* v3z

     #Null sequence  component.
     E0 = cm.polar((v1z.real+ v2z.real+  v3z.real)/3.0 +  (v1z.imag+ v2z.imag+ v3z.imag)/3.0*1j)

     #Positive sequence component.
     E1 = cm.polar((v1z.real + av2.real +  a2v3.real)/3.0+ (v1z.imag+ av2.imag+ a2v3.imag)/3.0*1j)

     #Negative sequence component.
     E2 = cm.polar((v1z.real + a2v2.real +  av3.real)/3.0+ (v1z.imag+ a2v2.imag+ av3.imag)/3.0*1j)

     return (E0, E1, E2)


def  symcompz(v1, v2, v3):
     """
     Returns the symmetrical components of the three phasors v1, v2, v3
     which are in tuple (r, theta) form.
     """
     av2  = _a_ * v2
     a2v2 = _a2_ * v2
     av3  = _a_* v3
     a2v3 = _a2_* v3

     #Null sequence  component.
     E0 = polar((v1.real+ v2.real+  v3.real)/3.0 +  (v1.imag+ v2.imag+ v3.imag)/3.0*1j)

     #Positive sequence component.
     E1 = polar((v1.real + av2.real +  a2v3.real)/3.0+ (v1.imag+ av2.imag+ a2v3.imag)/3.0*1j)

     #Negative sequence component.
     E2 = polar((v1.real + a2v2.real +  av3.real)/3.0+ (v1.imag+ a2v2.imag+ av3.imag)/3.0*1j)

     return (E0, E1, E2)

def  symcomp2phasors(E0, E1, E2):
     """
     Recreates the phasors form the symmetrical components.
     """
     V1 = cm.polar(cm.rect(E0[0], E0[1]) + cm.rect(E1[0], E1[1]) + cm.rect(E2[0], E2[1]))
     V2 = cm.polar(cm.rect(E0[0], E0[1]) + _a2_* cm.rect(E1[0], E1[1]) + _a_ *cm.rect(E2[0], E2[1]))
     V3 = cm.polar(cm.rect(E0[0], E0[1]) + _a_* cm.rect(E1[0], E1[1]) + _a2_ * cm.rect(E2[0], E2[1]))
     return V1, V2, V3


if __name__ == "__main__":
   #extreme cases.
   I1 = cm.polar(10)
   I2 = cm.polar(0)
   I3 = cm.polar(0)
   print (symcomp(I1, I2, I3))
  
  

   #extreme cases, balanced system.
   i1 = 1
   i2 = -0.5 + mt.sqrt(3)/2.0 * 1j
   i3 = -0.5 - mt.sqrt(3)/2.0 * 1j

   I1 = cm.polar(i1) 
   I2 = cm.polar(i2)
   I3 = cm.polar(i3)
   E0, E1, E2 = symcomp(I1, I2, I3)   
   print ("original phasors=", I1, I2, I3)
   print ("symmetrical components:", E0, E1, E2)
   phasors = symcomp2phasors(E0, E1, E2)
   print ("recovered phasors:", phasors)
   #include more here! from published books or other sources.


((3.3333333333333335, 0.0), (3.3333333333333335, 0.0), (3.3333333333333335, 0.0))
original phasors= (1.0, 0.0) (1.0, 2.0943951023931957) (1.0, -2.0943951023931957)
symmetrical components: (1.4802973661668753e-16, 3.141592653589793) (7.401486830834377e-17, 0.0) (1.0, 0.0)
recovered phasors: ((0.9999999999999999, 1.812781560154486e-32), (0.9999999999999999, 2.0943951023931953), (0.9999999999999999, -2.0943951023931953))


### Creamos un módulo symmcomponent.py 

Donde ponemos todas las funciones que sirven para operar sobre fasores

In [3]:
import symmcomponent as sc
import math as mt
import cmath as cm


if __name__ == "__main__":
   #extreme cases.
   I1 = cm.polar(10)
   I2 = cm.polar(0)
   I3 = cm.polar(0)
   print (sc.symcomp(I1, I2, I3))
  
  

   #extreme cases, balanced system.
   i1 = 1
   i2 = -0.5 + mt.sqrt(3)/2.0 * 1j
   i3 = -0.5 - mt.sqrt(3)/2.0 * 1j

   I1 = cm.polar(i1) 
   I2 = cm.polar(i2)
   I3 = cm.polar(i3)
   E0, E1, E2 = sc.symcomp(I1, I2, I3)   
   print ("original phasors=", I1, I2, I3)
   print ("symmetrical components:", E0, E1, E2)
   phasors = sc.symcomp2phasors(E0, E1, E2)
   print ("recovered phasors:", phasors)

((3.3333333333333335, 0.0), (3.3333333333333335, 0.0), (3.3333333333333335, 0.0))
original phasors= (1.0, 0.0) (1.0, 2.0943951023931957) (1.0, -2.0943951023931957)
symmetrical components: (1.4802973661668753e-16, 3.141592653589793) (7.401486830834377e-17, 0.0) (1.0, 0.0)
recovered phasors: ((0.9999999999999999, 1.812781560154486e-32), (0.9999999999999999, 2.0943951023931953), (0.9999999999999999, -2.0943951023931953))


## Lista de argumentos *args y **kwargs

In [10]:
#
# *args, lista de argumentos sin clave
#
def test_var_args(f_arg, *argv):
    print ("first normal arg:", f_arg)
    for arg in argv:
        print ("another arg through *argv :", arg)

test_var_args('yasoob','python','eggs','test')

first normal arg: yasoob
another arg through *argv : python
another arg through *argv : eggs
another arg through *argv : test


In [11]:
#
# **kwars, lista de argumentos con clave
#

def greet_me(**kwargs):
    if kwargs is not None:
        print (kwargs)
        for key, value in kwargs.items():
            print ("%s == %s" %(key,value))

    if 'name' in kwargs.keys():
       print ('name: ', kwargs['name'])


    if 'age' in kwargs.keys():
       print ('age: ', kwargs['age'])


greet_me(name="yasoob", age= 45)

{'age': 45, 'name': 'yasoob'}
age == 45
name == yasoob
name:  yasoob
age:  45


## Grabar y Recuperar objetos: serialización de objetos

Más información [acá](https://docs.python.org/2/library/pickle.html)

In [31]:
import pickle
a = [1,2,3,4]
b = "misVariables"
c = 123

D = { "a": a, "b": b}

pickfilename = "./tmp.pck"

with open(pickfilename, 'wb') as f:   
    pickle.dump(D, f)

with open(pickfilename, 'rb') as f:
    myvars = pickle.load(f)

print (myvars)

{'a': [1, 2, 3, 4], 'b': 'misVariables'}


## Bases de datos orientadas a documentos
### MongoDB [link](https://www.mongodb.org/)

Es una base de datos no relacional con esquemas flexibles en la que no se necesita definir la estructura de datos antes de insertar datos en la base y que soporta métodos de busqueda por claves.

In [ ]:
#
# para insertar datos en la base
#
misDocumentos.insert(D)

#
# para recuperar datos de la base
#
doc = misDocumentos.find_one ()

#
# query
#
doc = misDocumentos.find({"b": "misVariables"})

## Decoradores [link](http://thecodeship.com/patterns/guide-to-python-function-decorators/)

In [1]:
# Las funciones son objetos

def greet1(name):
    return "hello " + name

# asignar funciones a variables
greet_someone = greet1
print (greet_someone("John"))

hello John


In [2]:
#definir funciones dentro de funciones

def greet2(name):
    
    def get_message():
        return "Hello "

    result = get_message() + name
    return result

print (greet2("John"))

Hello John


In [3]:
#Las funciones pueden ser pasadas como 
#parámetros a otras funciones

def greet3(name):
   return "Hello " + name

def call_func(func):
    other_name = "John"
    return func(other_name)

print (call_func(greet3))

Hello John


In [4]:
# Las funciones pueden retornar otras funciones

def compose_greet_func():
    def get_message():
        return "Hello there!"

    return get_message

greet = compose_greet_func()
# greet es una función
print (greet())

Hello there!


In [5]:
# las funciones internas a las funciones
# tienen acceso al alcance interno

def compose_greet_func(name):
    b = "***"
    def func1 ():
        return b + " -- "
    def get_message():
        a = func1()
        return a + "Hello there "+name+"!"

    return get_message

greet = compose_greet_func("John")
print (greet())

*** -- Hello there John!


## Decoradores

In [ ]:
#
# Decoradores
#
def get_text(name):
   return "lorem ipsum, {0} dolor sit amet".format(name)

def p_decorate(func):
   def func_wrapper(name):
       return "<p>{0}</p>".format(func(name))
   return func_wrapper

# Un decorador
my_get_text = p_decorate(get_text)

print (my_get_text("John"))

In [6]:
#
#  Sintáxis para decoradores en python
#
def p_decorate1(func):
   def func_wrapper(name):
       return "<p>{0}</p>".format(func(name))
   return func_wrapper

@p_decorate1
def get_text2(name):
   return "lorem ipsum, {0} dolor sit amet".format(name)

print (get_text2("John"))

<p>lorem ipsum, John dolor sit amet</p>


In [7]:
#
# Ejemplo de aplicar 3 decoradores 
#

def p_decorate2(func):
   def func_wrapper(name):
       return "<p>{0}</p>".format(func(name))
   return func_wrapper

def strong_decorate(func):
    def func_wrapper(name):
        return "<strong>{0}</strong>".format(func(name))
    return func_wrapper

def div_decorate(func):
    def func_wrapper(name):
        return "<div>{0}</div>".format(func(name))
    return func_wrapper


@div_decorate
@p_decorate2
@strong_decorate
def get_text3(name):
   return "lorem ipsum, {0} dolor sit amet".format(name)

print ("Ejemplo de 3 decoradores")

print (get_text3("John"))

Ejemplo de 3 decoradores
<div><p><strong>lorem ipsum, John dolor sit amet</strong></p></div>


In [8]:
#
# métodos decoradores
# en python los métodos son funciones en el que primer
# parámetro es una referencia al primer objeto
#

def p_decorate3(func):
   def func_wrapper(self):
       return "<p>{0}</p>".format(func(self))
   return func_wrapper

class Person(object):
    def __init__(self):
        self.name = "John"
        self.family = "Doe"

    @p_decorate3
    def get_fullname(self):
        return self.name+" "+self.family

print ("Ejemplo con clase")

my_person = Person()
print (my_person.get_fullname())

Ejemplo con clase
<p>John Doe</p>
